In [1]:
import os, re, time, json
import datetime
import PIL.Image, PIL.ImageFont, PIL.ImageDraw
import numpy as np
import tensorflow as tf
import google
 
from matplotlib import pyplot as plt
print("Tensorflow version " + tf.__version__)

Tensorflow version 2.3.1


In [2]:
import tensorflow as tf
import tensorflow.keras as keras

from src.generators.tf_records_generator import get_dataset
from src.models.efficientNet_B0 import EffNet0, unfreeze_blocks
from src.utils.log_files import save_config, save_fold_iter_history,get_log_dir
from src.utils.kfold import get_kfold_split

In [3]:
CONFIG = dict(
    effnet_version = 0,
    input_shape=(256, 256, 3),
    image_resolution=256,
    trainable_base=False,
    time=datetime.datetime.now().strftime("_%d_%m_%Y_%H_%M"),
    use_patient_data=True,
    
    replicas=1, 
    batch_size=8,
    epochs=5,
    optimizer='adam',
    loss='binary_crossentropy',
    
    output_bias=np.log([584/32542]),
    weight_for_0 = (1 / 32542)*(32542+584)/2.0,
    weight_for_1 = (1 / 584)*(32542+584)/2.0
)


In [4]:
IS_COLAB_BACKEND = 'COLAB_GPU' in os.environ  # this is always set on Colab, the value is 0 or 1 depending on GPU presence
if IS_COLAB_BACKEND:
  from google.colab import auth
  # Authenticates the Colab machine and also the TPU using your
  # credentials so that they can access your private GCS buckets.
  auth.authenticate_user()
  data_dir = 'gs://dataset_files/'
else:
  data_dir = 'dataset/'

In [5]:
# Detect hardware
try:
  tpu_resolver = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError:
  tpu_resolver = None
  gpus = tf.config.experimental.list_logical_devices("GPU")

# Select appropriate distribution strategy
if tpu_resolver:
  tf.config.experimental_connect_to_cluster(tpu_resolver)
  tf.tpu.experimental.initialize_tpu_system(tpu_resolver)
  strategy = tf.distribute.TPUStrategy(tpu_resolver)
  print('Running on TPU ', tpu_resolver.cluster_spec().as_dict()['worker'])
elif len(gpus) > 1:
  strategy = tf.distribute.MirroredStrategy([gpu.name for gpu in gpus])
  print('Running on multiple GPUs ', [gpu.name for gpu in gpus])
elif len(gpus) == 1:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on single GPU ', gpus[0].name)
else:
  strategy = tf.distribute.get_strategy() # default strategy that works on CPU and single GPU
  print('Running on CPU')
  
print("Number of accelerators: ", strategy.num_replicas_in_sync)
CONFIG["replicas"] = strategy.num_replicas_in_sync


Running on single GPU  /device:GPU:0
Number of accelerators:  1


In [6]:
path = data_dir + str(CONFIG["image_resolution"])
files_train = np.sort(np.array(tf.io.gfile.glob(path + '/train*.tfrec')))
files_test = np.sort(np.array(tf.io.gfile.glob(path + '/test*.tfrec')))


ds = get_dataset(files_train[:-3], CONFIG)
val_ds = get_dataset(files_train[-3], CONFIG)
test_ds = get_dataset(files_test, CONFIG)

In [7]:
def make_model(config:dict, strategy)    :
    with strategy.scope():
      model = EffNet0(config=config, trainable_base=config['trainable_base'])
      model.compile(optimizer=config["optimizer"], loss=config["loss"], 
                    metrics=[
                                  keras.metrics.TruePositives(name='tp'),
                                  keras.metrics.FalsePositives(name='fp'),
                                  keras.metrics.TrueNegatives(name='tn'),
                                  keras.metrics.FalseNegatives(name='fn'), 
                                  keras.metrics.BinaryAccuracy(name='accuracy'),
                                  keras.metrics.Precision(name='precision'),
                                  keras.metrics.Recall(name='recall'),
                                  keras.metrics.AUC(name='auc'),
                            ]
    )
    return model


In [8]:
kfold_split = get_kfold_split(files_train)
save_config(CONFIG)
for iter, files in kfold_split.items():
    train_ds = get_dataset(files['train'],CONFIG,patient_info=True) 
    val_ds = get_dataset(files['validation'], CONFIG,patient_info=True)
    
    model = make_model(CONFIG, strategy=strategy)    
    
    history = model.fit(train_ds,class_weight={0:CONFIG["weight_for_0"],1:CONFIG["weight_for_1"]},
                        validation_data=val_ds,validation_batch_size=CONFIG["batch_size"]*CONFIG["replicas"],
                        batch_size=CONFIG["batch_size"]*CONFIG["replicas"], epochs=CONFIG["epochs"])
    
    save_fold_iter_history(CONFIG, history.history,iter)
    

(<tf.Tensor 'args_0:0' shape=() dtype=string>, (<tf.Tensor 'args_1:0' shape=() dtype=int64>, <tf.Tensor 'args_2:0' shape=() dtype=int64>, <tf.Tensor 'args_3:0' shape=() dtype=int64>))
Tensor("args_0:0", shape=(), dtype=string)
(<tf.Tensor 'args_1:0' shape=() dtype=int64>, <tf.Tensor 'args_2:0' shape=() dtype=int64>, <tf.Tensor 'args_3:0' shape=() dtype=int64>)
(<tf.Tensor 'args_0:0' shape=() dtype=string>, (<tf.Tensor 'args_1:0' shape=() dtype=int64>, <tf.Tensor 'args_2:0' shape=() dtype=int64>, <tf.Tensor 'args_3:0' shape=() dtype=int64>))
Tensor("args_0:0", shape=(), dtype=string)
(<tf.Tensor 'args_1:0' shape=() dtype=int64>, <tf.Tensor 'args_2:0' shape=() dtype=int64>, <tf.Tensor 'args_3:0' shape=() dtype=int64>)
Epoch 1/5
3269/3269 [==============================] - 809s 247ms/step - loss: 0.8899 - tp: 270.0000 - fp: 8714.0000 - tn: 16973.0000 - fn: 195.0000 - accuracy: 0.6593 - precision: 0.0301 - recall: 0.5806 - auc: 0.6612 - val_loss: 0.6588 - val_tp: 71.0000 - val_fp: 2193.000

KeyboardInterrupt: 

In [8]:
model = make_model(CONFIG, strategy=strategy)  

In [9]:
model.layers[1].trainable

False

In [11]:
from tensorflow.keras.models import Model
output_bias = keras.initializers.Constant(CONFIG['output_bias'])


input_img = keras.layers.Input(shape=(256, 256, 3))
conv_base = keras.applications.efficientnet.EfficientNetB0(weights='imagenet', include_top=False,
                                                               input_shape=(256,256,3))(input_img)
conv_base.trainable = False
gmp = keras.layers.GlobalMaxPooling2D(name="gap")(conv_base)
dense1 = keras.layers.Dense(1024, activation='relu')(gmp)
dropout1 = keras.layers.Dropout(0.4)(dense1)
fc_out = keras.layers.Dense(1, activation="sigmoid", name="fc_out", bias_initializer=output_bias)(dropout1)
model = Model(inputs=input_img,outputs=fc_out)

In [12]:
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 256, 256, 3)]     0         
_________________________________________________________________
efficientnetb0 (Functional)  (None, 8, 8, 1280)        4049571   
_________________________________________________________________
gap (GlobalMaxPooling2D)     (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              1311744   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
fc_out (Dense)               (None, 1)                 1025      
Total params: 5,362,340
Trainable params: 5,320,317
Non-trainable params: 42,023
_______________________________________

In [1]:
import tensorflow as tf
IMG_SHAPE = (256, 256, 3)
model0 = tf.keras.applications.EfficientNetB0(input_shape=(256,256,3),include_top=False, weights="imagenet")

In [1]:
model0.summary()

NameError: name 'model0' is not defined

In [77]:
model0.layers[122].strides

(1, 1)

In [5]:
model = EffNet0(config=CONFIG, trainable_base=False)

In [6]:
model.layers[0].layers[-2].trainable




False